In [2]:
from __future__ import division

import numpy as np
import os, sys
from PIL import Image
import pandas as pd
import json
import pickle
import ast
import uuid

from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

from IPython.core.pylabtools import figsize, getfigs

import seaborn as sns
from sklearn.model_selection import StratifiedKFold

import random
from functools import reduce

from scipy.stats import norm
from IPython.display import clear_output

import copy
import importlib

# import urllib library
from urllib.request import urlopen

### Add Paths

## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..')) ## u\e relative paths

## add helpers to python path
import sys
if os.path.join(proj_dir, 'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'stimuli'))

## Read in instructions from paired_building

In [3]:
df_chat = pd.read_csv('../../analysis/results/csv/df_chat_cogsci21.csv')
df_trial = pd.read_csv('../../analysis/results/csv/df_trial_cogsci21.csv')

In [4]:
# exclude inaccurate ppts

# 75% Accuracy on 75% of trials
df75 = pd.DataFrame(df_trial.groupby(['gameid', 'trialNum'])['trialScore'].sum()>75).groupby(['gameid']).sum()
df75['trials'] = df75['trialScore']

df75 = df75[df75['trials']>=9]
includedGames = list(df75.reset_index().gameid)

print("Total dyads achieving 75% Accuracy on 75% of trials:",len(df75))

df_chat = df_chat[df_chat.gameid.isin(includedGames)]

Total dyads achieving 75% Accuracy on 75% of trials: 49


In [5]:
df_instructions = df_chat[~df_chat.practice].reset_index(drop=True)

In [6]:
random.seed(0)
df_instructions['message_id'] = [str(uuid.UUID(int=random.getrandbits(128))) for row in df_instructions.iloc[:,0]]

In [7]:
df_instructions.to_csv('../../analysis/results/csv/df_chat_ids_cogsci21.csv')

In [8]:
# setup
gameids = list(df_instructions.gameid.unique())
trialNums = list(df_instructions.trialNum.unique())

def unformat(message):
    message = message.replace("~~~",".")
    return message

In [9]:
df_instructions.loc[:,'message_num'] = (df_instructions.turnNum / 2).astype(int)
df_instructions.loc[:,'clean_msg'] = df_instructions['content'].apply(lambda message: unformat(message))

In [10]:
# An entire game is a single metadatum

metadata = []

# iterate over gameid
for gameid in gameids:
    
    df_ppt = df_instructions[df_instructions.gameid == gameid]
    ppt_row = df_ppt.iloc[0,:]
    
    datum = {
        'gameid' : ppt_row.gameid,
        'iterationName': ppt_row.iterationName,
        'games' : [],
        'workerIDs' : [],
        'numGames' : 0,
        'trials' : []
    }
    
    # iterate over trials
    for trialNum in trialNums:
        
        df_whole_trial = df_ppt[df_ppt.trialNum == trialNum].reset_index()
        trial_row = df_whole_trial.iloc[0,:]

        # save all messages in dict
#         messages = df_whole_trial['content'].apply(lambda message: unformat(message)).to_dict()
#         messages = {str(k): v for k,v in messages.items()}

        messages = df_whole_trial[['message_num','clean_msg','message_id']].to_dict(orient='records')
        
        trial_datum = \
            {
            'trialNum' : int(trialNum),
            'messages' : messages,
            'leftTarget' : trial_row.leftTarget,
            'rightTarget' : trial_row.rightTarget,
            'scene' : [trial_row.leftTarget, trial_row.rightTarget],
#             'gameid': trial_row.gameid,
            'iterationName': trial_row.gameid,
            }
        
        datum['trials'].append(trial_datum)
        
    metadata.append(datum)

In [11]:
metadata[0]

{'gameid': '1494-029e2297-bd3f-4cfe-be00-d06d36724e4e',
 'iterationName': 'pilot0',
 'games': [],
 'workerIDs': [],
 'numGames': 0,
 'trials': [{'trialNum': 0,
   'messages': [{'message_num': 0,
     'clean_msg': 'put a blue block three from the left. ',
     'message_id': 'e3e70682-c209-4cac-629f-6fbed82c07cd'},
    {'message_num': 1,
     'clean_msg': 'place a red block on top of the left side of the blue block and then another red on top of that red',
     'message_id': 'f728b4fa-4248-5e3a-0a5d-2f346baa9455'},
    {'message_num': 2,
     'clean_msg': 'Now place a blue block on top of that that lines up with the bottom blue block',
     'message_id': 'eb1167b3-67a9-c378-7c65-c1e582e2e662'},
    {'message_num': 3,
     'clean_msg': 'remove that last blue block, place it on top of the red blocks',
     'message_id': 'f7c1bd87-4da5-e709-d471-3d60c8a70639'},
    {'message_num': 4,
     'clean_msg': 'place a red block on the second grid from the right of the screen\n',
     'message_id': 

In [12]:
len(metadata)

49

In [13]:
stim_col_name = 'ca_ref_exp'

### internal annotations, workerID version

Add a record for each workerID

In [14]:
# use appWid.js and storeWid.js!

stim_col_name = 'ca_ref_exp_internal'

In [15]:
workers = ['zoe','charles','test']

In [16]:
from math import ceil

In [17]:
range(batch*batch_size, batch*(batch_size+1))

NameError: name 'batch' is not defined

In [18]:

n_batches = 10

n_gameids = len(gameids)

batch_size = ceil(n_gameids/n_batches)

In [19]:
batch_size

5

In [20]:
# each metadatum contains len(gameids)/batch_size gameids

n_batches = 10

n_gameids = len(gameids)

batch_size = ceil(n_gameids/n_batches)

metadata = []

for worker in workers:
    # iterate over gameid
    for batch in range(0, n_batches):
        
        datum = {
                'batch': batch,
                'iterationName': ppt_row.iterationName,
                'workerID' : worker,
                'attempts': 0,
                'trials' : []
            }
        
        for gameidx in range(batch*batch_size, (batch+1)*batch_size):
            
            if gameidx < n_gameids:
                gameid = gameids[gameidx]

                df_ppt = df_instructions[df_instructions.gameid == gameid]
                ppt_row = df_ppt.iloc[0,:]

                # iterate over trials
                for trialNum in trialNums:

                    df_whole_trial = df_ppt[df_ppt.trialNum == trialNum].reset_index()
                    trial_row = df_whole_trial.iloc[0,:]

                    # save all messages in dict
            #         messages = df_whole_trial['content'].apply(lambda message: unformat(message)).to_dict()
            #         messages = {str(k): v for k,v in messages.items()}

                    messages = df_whole_trial[['message_num','clean_msg','message_id']].to_dict(orient='records')

                    trial_datum = \
                        {
                        'dyad_gameid' : ppt_row.gameid,
                        'batch' : batch,
                        'trialNum' : int(trialNum),
                        'messages' : messages,
                        'leftTarget' : trial_row.leftTarget,
                        'rightTarget' : trial_row.rightTarget,
                        'scene' : [trial_row.leftTarget, trial_row.rightTarget],
                        'iterationName': trial_row.gameid
                        }

                    datum['trials'].append(trial_datum)

        metadata.append(datum)

In [21]:
len(metadata)

30

In [208]:
# connect to mongo

import pymongo as pm

# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## experiment server ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[stim_col_name]

In [209]:
# Clear metadata collection

really_run = True;

if really_run:
    db.drop_collection(stim_col_name)

In [210]:
## now, iterate through each version and insert into mongo
## loop through list of records and insert each into collection
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(metadata):
        coll.insert_one(j)
        print('Inserted version of stimDict.')
        clear_output(wait=True)

else:
    print('Did not insert any new data.')

Inserted version of stimDict.


In [211]:
records = list(coll.find())

In [212]:
len(records)

30

In [197]:
gameids[10:]

['3086-e9f8b532-4969-4703-852e-f6f029d281e2',
 '4909-705b801e-4ec3-4910-b180-bb7612d80f25',
 '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8',
 '6753-24a4a6e7-e017-4cba-a068-8dea7af05cf2',
 '5809-2a5e096b-a425-4bc7-b495-cf74cb60ded6',
 '6598-80d65299-0654-43b4-9f27-08572427cb2d',
 '0208-b497c845-b076-45c9-b958-a62a3ae6e65f',
 '0741-0d5c3f38-e1fa-4a05-9f35-4a81c7cdbd71',
 '4244-cb63e4ee-2692-4273-8dcb-45c1cde2496a',
 '4548-d4f8915b-8223-4cc4-b5b4-889e40eb849f',
 '4978-4c585194-1823-46fc-989c-e6200c5d61e6',
 '6064-78a665ff-7528-45c6-91d1-960cc9647fa6',
 '6169-888a2828-0e46-483c-b48d-cf18271187ed',
 '6210-0e17e8cb-6041-4783-af4a-0da47b21b402',
 '8263-001a1b6c-ab69-48aa-8807-752d70fb55cf',
 '9018-fbca1193-d7bc-4044-809c-f7c21bdcffe8',
 '9961-1bd92164-cbe7-4841-8225-7de8486bf84a',
 '0110-5784fec9-109a-4d7a-a343-4820f4d42144',
 '0738-513adaa7-2548-44a4-8d01-7e2fb3ecbfd4',
 '1690-8f854c73-420c-4820-b76f-2c2ac28a3035',
 '2409-8e3cd70d-3066-4782-9d23-e3fecd179ac1',
 '2708-886eba61-2274-429a-bd7e-05b

In [131]:
records

[{'_id': ObjectId('642af34aa35a2680836172b3'),
  'dyad_gameid': '1494-029e2297-bd3f-4cfe-be00-d06d36724e4e',
  'iterationName': 'pilot0',
  'workerID': 'zoe',
  'attempts': 0,
  'trials': [{'trialNum': 0,
    'messages': [{'message_num': 0,
      'clean_msg': 'put a blue block three from the left. ',
      'message_id': 'e3e70682-c209-4cac-629f-6fbed82c07cd'},
     {'message_num': 1,
      'clean_msg': 'place a red block on top of the left side of the blue block and then another red on top of that red',
      'message_id': 'f728b4fa-4248-5e3a-0a5d-2f346baa9455'},
     {'message_num': 2,
      'clean_msg': 'Now place a blue block on top of that that lines up with the bottom blue block',
      'message_id': 'eb1167b3-67a9-c378-7c65-c1e582e2e662'},
     {'message_num': 3,
      'clean_msg': 'remove that last blue block, place it on top of the red blocks',
      'message_id': 'f7c1bd87-4da5-e709-d471-3d60c8a70639'},
     {'message_num': 4,
      'clean_msg': 'place a red block on the secon

### Top-up (i.e. refill missing data)

In [34]:
datumA = [datum for datum in metadata if (datum['workerID'] == "zoe") & (datum['batch'] in [2.0])][0]
datumA['trials'] = [trial for trial in datumA['trials'] if trial['dyad_gameid'] == '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8']
datumA

{'batch': 2,
 'iterationName': 'pilot2',
 'workerID': 'zoe',
 'attempts': 0,
 'trials': [{'dyad_gameid': '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8',
   'batch': 2,
   'trialNum': 0,
   'messages': [{'message_num': 0,
     'clean_msg': 'place 2 blue blocks on the left on either side of the red mark and then 2 red blocks on top on left',
     'message_id': 'ecf150e6-c6f1-49c7-0c15-a57509c64c1d'},
    {'message_num': 1,
     'clean_msg': 'move red on top of red. Then add 1 blue on right splitting the red hash and stack 2 red on top',
     'message_id': '25cf51c4-9c89-9296-92b0-35e01ca5a2f5'}],
   'leftTarget': 'L',
   'rightTarget': 'C',
   'scene': ['L', 'C'],
   'iterationName': '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8'},
  {'dyad_gameid': '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8',
   'batch': 2,
   'trialNum': 1,
   'messages': [{'message_num': 0,
     'clean_msg': '2 blue on either side of left red mark then 2 red on top of each other on left side.',
     'message_id': 'c96ef1a5-

In [33]:
datumB = [datum for datum in metadata if (datum['workerID'] == "zoe") & (datum['batch'] in [3.0])][0]
datumB['trials'] = [trial for trial in datumB['trials'] if trial['dyad_gameid'] == '0208-b497c845-b076-45c9-b958-a62a3ae6e65f']
datumB

{'batch': 3,
 'iterationName': 'pilot3',
 'workerID': 'zoe',
 'attempts': 0,
 'trials': [{'dyad_gameid': '0208-b497c845-b076-45c9-b958-a62a3ae6e65f',
   'batch': 3,
   'trialNum': 0,
   'messages': [{'message_num': 0,
     'clean_msg': 'blue 2,1',
     'message_id': '8d00b7b2-7d82-fccf-81eb-88e5d549faad'},
    {'message_num': 1,
     'clean_msg': 'blue 4,1\n',
     'message_id': '43eb280e-7901-7990-64e3-22cfa51de78b'},
    {'message_num': 2,
     'clean_msg': 'blue 9,1\n',
     'message_id': '5d443821-8a7f-03ae-3ad2-3fbf2e573864'},
    {'message_num': 3,
     'clean_msg': 'red 2,2',
     'message_id': '9bde299d-da28-6b7a-4d6d-99a628d798b4'},
    {'message_num': 4,
     'clean_msg': 'red 2, 4',
     'message_id': '74f8c57b-24c5-82ca-eab3-c4d8d4e4004b'},
    {'message_num': 5,
     'clean_msg': 'red 9,2',
     'message_id': '7a658abe-1188-afa1-f660-ffd5119fc1c4'},
    {'message_num': 6,
     'clean_msg': 'red 9,4',
     'message_id': '68f49b51-f3fa-8613-90ed-41f464aba3f6'},
    {'message

In [38]:
[datumA, datumB]

[{'batch': 2,
  'iterationName': 'pilot2',
  'workerID': 'zoe',
  'attempts': 0,
  'trials': [{'dyad_gameid': '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8',
    'batch': 2,
    'trialNum': 0,
    'messages': [{'message_num': 0,
      'clean_msg': 'place 2 blue blocks on the left on either side of the red mark and then 2 red blocks on top on left',
      'message_id': 'ecf150e6-c6f1-49c7-0c15-a57509c64c1d'},
     {'message_num': 1,
      'clean_msg': 'move red on top of red. Then add 1 blue on right splitting the red hash and stack 2 red on top',
      'message_id': '25cf51c4-9c89-9296-92b0-35e01ca5a2f5'}],
    'leftTarget': 'L',
    'rightTarget': 'C',
    'scene': ['L', 'C'],
    'iterationName': '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8'},
   {'dyad_gameid': '5895-de8412be-4cb1-49e4-9c61-ca6a11481af8',
    'batch': 2,
    'trialNum': 1,
    'messages': [{'message_num': 0,
      'clean_msg': '2 blue on either side of left red mark then 2 red on top of each other on left side.',
      '

In [45]:
metadata = [datumA, datumB]

In [46]:
stim_col_name = 'ca_ref_exp_internal_topup'

In [52]:
# connect to mongo

import pymongo as pm

# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## experiment server ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']

stim_col_name
coll = db[stim_col_name]

In [53]:
# Clear metadata collection

really_run = True;

if really_run:
    db.drop_collection(stim_col_name)

In [54]:
## now, iterate through each version and insert into mongo
## loop through list of records and insert each into collection
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(metadata):
        coll.insert_one(j)
        print('Inserted version of stimDict.')
        clear_output(wait=True)

else:
    print('Did not insert any new data.')

Inserted version of stimDict.


In [55]:
len(list(coll.find()))

2

### OLD

In [111]:
s3_bucket_path_template = "https://lax-{}-{}-all.s3.amazonaws.com/"

In [113]:
subdomains = {
    'drawing' :  ['nuts-bolts','wheels','furniture','dials']
}

# domains = list(subdomains.keys())
domains = ['drawing']

### Load local data

In [114]:
# in development: this contains metadata for all context conditions for a single stimulus
# stimulus_data = json.load(open('./context-stimuli/lax-drawing-context-stimuli.json'))

In [115]:
# in development: this contains metadata for all context conditions for all subdomains
stimulus_data = json.load(open('./context-stimuli/lax-drawing-context-stimuli-all-subdomains.json'))

In [34]:
stimulus_data['stimuli']

[{'target': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png',
  'subdomain': 'furniture',
  'context_0': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-211.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-037.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-094.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-103.png'],
  'context_1': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-041.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-091.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-098.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-127.png'],
  'context_2': ['https://lax-drawing-nuts-bolts-all.s3.amazonaws.com/lax-drawing-nuts-bolts-all-161.png',
   'https:

In [35]:
pd.DataFrame.from_dict(stimulus_data['stimuli'])

,target,subdomain,context_0,context_1,context_2
0,https://lax-drawing-furniture-all.s3.amazonaws...,furniture,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-nuts-bolts-all.s3.amazona...
1,https://lax-drawing-furniture-all.s3.amazonaws...,furniture,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-nuts-bolts-all.s3.amazona...
2,https://lax-drawing-furniture-all.s3.amazonaws...,furniture,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-nuts-bolts-all.s3.amazona...
3,https://lax-drawing-furniture-all.s3.amazonaws...,furniture,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-furniture-all.s3.amazonaw...,[https://lax-drawing-nuts-bolts-all.s3.amazona...
4,https://lax-drawing-nuts-bolts-all.s3.amazonaw...,nuts_bolts,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-furniture-all.s3.amazonaw...
5,https://lax-drawing-nuts-bolts-all.s3.amazonaw...,nuts_bolts,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-furniture-all.s3.amazonaw...
6,https://lax-drawing-nuts-bolts-all.s3.amazonaw...,nuts_bolts,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-furniture-all.s3.amazonaw...
7,https://lax-drawing-nuts-bolts-all.s3.amazonaw...,nuts_bolts,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-nuts-bolts-all.s3.amazona...,[https://lax-drawing-furniture-all.s3.amazonaw...
8,https://lax-drawing-wheels-all.s3.amazonaws.co...,wheels,[https://lax-drawing-wheels-all.s3.amazonaws.c...,[https://lax-drawing-wheels-all.s3.amazonaws.c...,[https://lax-drawing-furniture-all.s3.amazonaw...
9,https://lax-drawing-wheels-all.s3.amazonaws.co...,wheels,[https://lax-drawing-wheels-all.s3.amazonaws.c...,[https://lax-drawing-wheels-all.s3.amazonaws.c...,[https://lax-drawing-furniture-all.s3.amazonaw...


In [36]:
# construct trial objects from stimulus data

all_trials = []

for stimulus_dict in stimulus_data['stimuli']:
    
    target_URL = stimulus_dict['target']
    
    stimulus_conditions = [condition for condition in stimulus_dict.keys() if ((condition != 'target') and (condition != 'subdomain'))]
    
    for condition in stimulus_conditions:
        
        trial = {
            'target_URL' : target_URL,
            'target_domain' : target_URL.split('-')[1],
            'target_subdomain' : target_URL.split('-')[2],
            'target_id' : target_URL.split('-')[-1].split('.')[0],
            'context_condition' : condition,
            'context_items' : stimulus_dict[condition],
        }
        
        trial['target_subdomain'] = trial['target_subdomain'] if trial['target_subdomain'] != 'nuts' else 'nuts-bolts'
        
        all_trials.append(trial)
        
print(all_trials)

[{'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png', 'target_domain': 'drawing', 'target_subdomain': 'furniture', 'target_id': '217', 'context_condition': 'context_0', 'context_items': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-211.png', 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-037.png', 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-094.png', 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-103.png']}, {'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png', 'target_domain': 'drawing', 'target_subdomain': 'furniture', 'target_id': '217', 'context_condition': 'context_1', 'context_items': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-041.png', 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-091.png

In [37]:
all_trials

[{'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png',
  'target_domain': 'drawing',
  'target_subdomain': 'furniture',
  'target_id': '217',
  'context_condition': 'context_0',
  'context_items': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-211.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-037.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-094.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-103.png']},
 {'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png',
  'target_domain': 'drawing',
  'target_subdomain': 'furniture',
  'target_id': '217',
  'context_condition': 'context_1',
  'context_items': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-041.png',
   'https://lax-drawing-furniture-all.s3.amazonaws.com/

In [38]:
# create metadata for individual trials

metadata = []

for trial in all_trials:
    
    metadata = metadata + [{
        'domain': trial['target_domain'], # participant subdomain matches single trial subdomain
        'subdomain': trial['target_subdomain'], # participant subdomain matches single trial subdomain
        'trials': [trial],
        'games': [],
        'numGames': 0
    }]
    
metadata

[{'domain': 'drawing',
  'subdomain': 'furniture',
  'trials': [{'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png',
    'target_domain': 'drawing',
    'target_subdomain': 'furniture',
    'target_id': '217',
    'context_condition': 'context_0',
    'context_items': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-211.png',
     'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-037.png',
     'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-094.png',
     'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-103.png']}],
  'games': [],
  'numGames': 0},
 {'domain': 'drawing',
  'subdomain': 'furniture',
  'trials': [{'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png',
    'target_domain': 'drawing',
    'target_subdomain': 'furniture',
    'target_id': '217',
    'context_cond

In [39]:
len(metadata)

48

In [40]:
metadata[23]

{'domain': 'drawing',
 'subdomain': 'nuts-bolts',
 'trials': [{'target_URL': 'https://lax-drawing-nuts-bolts-all.s3.amazonaws.com/lax-drawing-nuts-bolts-all-208.png',
   'target_domain': 'drawing',
   'target_subdomain': 'nuts-bolts',
   'target_id': '208',
   'context_condition': 'context_2',
   'context_items': ['https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-134.png',
    'https://lax-drawing-wheels-all.s3.amazonaws.com/lax-drawing-wheels-all-082.png',
    'https://lax-drawing-dials-all.s3.amazonaws.com/lax-drawing-dials-all-221.png',
    'https://lax-drawing-dials-all.s3.amazonaws.com/lax-drawing-dials-all-206.png']}],
 'games': [],
 'numGames': 0}

#### Upload

In [49]:
stim_col_name = 'lax_context_single_trial_all_subdomains'

In [50]:
# connect to mongo

import pymongo as pm

# set vars 
auth = pd.read_csv('../../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## experiment server ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[stim_col_name]

In [51]:
# Clear metadata collection

really_run = True;

if really_run:
    db.drop_collection(stim_col_name)

In [52]:
## now, iterate through each version and insert into mongo
## loop through list of records and insert each into collection
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(metadata):
        coll.insert_one(j)
        print('Inserted version of stimDict.')
        clear_output(wait=True)

else:
    print('Did not insert any new data.')

Inserted version of stimDict.


In [60]:
records = list(coll.find())

In [62]:
[record for record in records if record['numGames']==0]

20

### top up

In [86]:
counts = pd.read_csv('../../analysis/context/item_counts_tmp.csv')
counts.stimulus = counts.stimulus.apply(lambda x: ast.literal_eval(x))

In [87]:
counts

,subdomain,stimulus,contextCondition,Unnamed: 0,_id_x,rt,context_items,responses,targetId,targetURL,...,iterationName_y,config_name_y,workerID_y,studyID_y,sessionID_y,shuffle,trialOrder,instructions,instruction_version,stimId
0,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_0,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_0,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_1,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_2,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_0,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
7,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_1,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
8,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
9,dials,https://lax-drawing-dials-all.s3.amazonaws.com...,context_0,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [88]:
trial = all_trials[0]

In [89]:
trial['target_URL']

'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png'

In [90]:
counts[(counts.subdomain == trial['target_subdomain']) &
       (counts.stimulus == trial['target_URL']) &
       (counts.contextCondition == trial['context_condition'])]

,subdomain,stimulus,contextCondition,Unnamed: 0,_id_x,rt,context_items,responses,targetId,targetURL,...,iterationName_y,config_name_y,workerID_y,studyID_y,sessionID_y,shuffle,trialOrder,instructions,instruction_version,stimId
21,furniture,https://lax-drawing-furniture-all.s3.amazonaws...,context_0,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [100]:
topup_metadata = []
n_expected = 3

for trial in all_trials:
    
    trial_counts = counts[(counts.subdomain == trial['target_subdomain']) &
       (counts.stimulus == trial['target_URL']) &
       (counts.contextCondition == trial['context_condition'])]
    
    assert (len(trial_counts) < 2)
    
    if len(trial_counts) > 0:
        trial_count = trial_counts.iloc[0]['context_items']
        left_to_go = max(0, 3 - trial_count)
    else:
        left_to_go = 0
        
        
    while left_to_go > 0:
        topup_metadata = topup_metadata + [{
            'domain': trial['target_domain'], # participant subdomain matches single trial subdomain
            'subdomain': trial['target_subdomain'], # participant subdomain matches single trial subdomain
            'trials': [trial],
            'games': [],
            'numGames': 0
        }]
        
        left_to_go -=1
    
topup_metadata

[{'domain': 'drawing',
  'subdomain': 'furniture',
  'trials': [{'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-217.png',
    'target_domain': 'drawing',
    'target_subdomain': 'furniture',
    'target_id': '217',
    'context_condition': 'context_2',
    'context_items': ['https://lax-drawing-nuts-bolts-all.s3.amazonaws.com/lax-drawing-nuts-bolts-all-161.png',
     'https://lax-drawing-wheels-all.s3.amazonaws.com/lax-drawing-wheels-all-189.png',
     'https://lax-drawing-dials-all.s3.amazonaws.com/lax-drawing-dials-all-135.png',
     'https://lax-drawing-dials-all.s3.amazonaws.com/lax-drawing-dials-all-196.png']}],
  'games': [],
  'numGames': 0},
 {'domain': 'drawing',
  'subdomain': 'furniture',
  'trials': [{'target_URL': 'https://lax-drawing-furniture-all.s3.amazonaws.com/lax-drawing-furniture-all-197.png',
    'target_domain': 'drawing',
    'target_subdomain': 'furniture',
    'target_id': '197',
    'context_condition': 'context_0',

In [101]:
len(topup_metadata)

27

In [106]:
stim_col_name = 'lax_context_single_trial_all_subdomains_topup'

In [107]:
# connect to mongo

import pymongo as pm

# set vars 
auth = pd.read_csv('../../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## experiment server ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[stim_col_name]

In [108]:
# Clear metadata collection

really_run = True;

if really_run:
    db.drop_collection(stim_col_name)

In [109]:
## now, iterate through each version and insert into mongo
## loop through list of records and insert each into collection
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(topup_metadata):
        coll.insert_one(j)
        print('Inserted version of stimDict.')
        clear_output(wait=True)

else:
    print('Did not insert any new data.')

Inserted version of stimDict.
